___
This notebook decides on the intermediate variables being used

In [2]:
import pandas as pd
import numpy as np
import scipy
from sklearn import *

/Users/shumingpeh/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/shumingpeh/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/shumingpeh/anaconda3/lib/python3.6/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be 

## Read data
- regularseason detailed results
- ~~cities~~
- teams
- coaches
    - there is a problem if a coach is new, so to prevent this from happening
    - coach will have a proxy variables of
        1. number of years of experience up to that year
        1. number of championship
        1. number of playoffs made

In [3]:
raw_data_regularseason = pd.read_csv("data/DataFiles/RegularSeasonDetailedResults.csv")

In [4]:
raw_data_teams = pd.read_csv("data/DataFiles/Teams.csv")

In [5]:
raw_data_coaches = pd.read_csv("data/DataFiles/TeamCoaches.csv")

In [6]:
raw_data_teams_coaches = (
    raw_data_teams
    .merge(raw_data_coaches, how='left', on=['TeamID'])
)

In [7]:
raw_data_regularseason.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [8]:
raw_data_regularseason.dtypes

Season      int64
DayNum      int64
WTeamID     int64
WScore      int64
LTeamID     int64
LScore      int64
WLoc       object
NumOT       int64
WFGM        int64
WFGA        int64
WFGM3       int64
WFGA3       int64
WFTM        int64
WFTA        int64
WOR         int64
WDR         int64
WAst        int64
WTO         int64
WStl        int64
WBlk        int64
WPF         int64
LFGM        int64
LFGA        int64
LFGM3       int64
LFGA3       int64
LFTM        int64
LFTA        int64
LOR         int64
LDR         int64
LAst        int64
LTO         int64
LStl        int64
LBlk        int64
LPF         int64
dtype: object

## Features to be included
- Season year
- winning/losing teamid
- winning/losing score
- winning/losing field goal percentage
- winning/losing field goal 3 point percentage
- winning/losing free throw percentage
- overall win rate

In [9]:
winning_teams_score_up_to_2013 = (
    raw_data_regularseason
    .pipe(lambda x:x.assign(winning_num_counts=1))
    .query("Season <= 2013")
    .groupby(['Season','WTeamID'])
    .agg({"WScore":"sum","WFGM":"sum","WFGA":"sum","WFGM3":"sum","WFGA3":"sum","WFTM":"sum","WFTA":"sum","LScore":"sum","winning_num_counts":"sum"})
    .reset_index()
    .rename(columns={"LScore":"losing_opponent_score"})
)

In [10]:
winning_teams_score_up_to_2013.head()

,Season,WTeamID,WScore,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,losing_opponent_score,winning_num_counts
0,2003,1102,825,271,480,120,259,163,249,638,12
1,2003,1103,1141,390,720,71,187,290,402,1019,13
2,2003,1104,1270,439,992,120,354,272,383,1046,17
3,2003,1105,556,179,433,64,157,134,180,465,7
4,2003,1106,888,322,700,76,207,168,270,753,13


In [11]:
losing_teams_score_up_to_2013 = (
    raw_data_regularseason
    .pipe(lambda x:x.assign(losing_num_counts=1))
    .query("Season <= 2013")
    .groupby(['Season','LTeamID'])
    .agg({"WScore":"sum","LScore":"sum","LFGM":"sum","LFGA":"sum","LFGM3":"sum","LFGA3":"sum","LFTM":"sum","LFTA":"sum","losing_num_counts":"sum"})
    .reset_index()
    .rename(columns={"WScore":"winning_opponent_score"})
)

In [12]:
losing_teams_score_up_to_2013.head()

,Season,LTeamID,winning_opponent_score,LScore,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,losing_num_counts
0,2003,1102,958,778,265,634,99,324,149,230,16
1,2003,1103,1091,986,343,788,76,247,224,296,14
2,2003,1104,774,670,234,609,58,202,144,203,11
3,2003,1105,1528,1310,455,1169,133,383,267,388,19
4,2003,1106,1032,893,334,848,95,287,130,191,15


In [13]:
combine_winning_losing_stats_for_year = (
    winning_teams_score_up_to_2013
    .merge(losing_teams_score_up_to_2013, how='left',left_on=['Season','WTeamID'],right_on=['Season','LTeamID'])
    .pipe(lambda x:x.assign(total_score = x.WScore + x.LScore))
    .pipe(lambda x:x.assign(total_opponent_score = x.winning_opponent_score + x.losing_opponent_score))
    .pipe(lambda x:x.assign(total_fgm = x.WFGM + x.LFGM))
    .pipe(lambda x:x.assign(total_fga = x.WFGA + x.LFGA))
    .pipe(lambda x:x.assign(total_fg3m = x.WFGM3 + x.LFGM3))
    .pipe(lambda x:x.assign(total_fg3a = x.WFGA3 + x.LFGA3))
    .pipe(lambda x:x.assign(total_ftm = x.WFTM + x.LFTM))
    .pipe(lambda x:x.assign(total_fta = x.WFTA + x.LFTA))
    .pipe(lambda x:x.assign(win_rate = x.winning_num_counts/(x.winning_num_counts + x.losing_num_counts)))
    .sort_values(['WTeamID','Season'])
)

In [27]:
combine_winning_losing_stats_for_year.head()
combine_winning_losing_stats_for_year.dtypes

Season                      int64
WTeamID                     int64
WScore                      int64
WFGM                        int64
WFGA                        int64
WFGM3                       int64
WFGA3                       int64
WFTM                        int64
WFTA                        int64
losing_opponent_score       int64
winning_num_counts          int64
LTeamID                     int64
winning_opponent_score      int64
LScore                      int64
LFGM                        int64
LFGA                        int64
LFGM3                       int64
LFGA3                       int64
LFTM                        int64
LFTA                        int64
losing_num_counts           int64
total_score                 int64
total_opponent_score        int64
total_fgm                   int64
total_fga                   int64
total_fg3m                  int64
total_fg3a                  int64
total_ftm                   int64
total_fta                   int64
win_rate      

In [15]:
cumulative_stats_for_team_each_year = (
    combine_winning_losing_stats_for_year
    .sort_values(['WTeamID','Season'])
    .groupby(['WTeamID'])
    .cumsum()
    .pipe(lambda x:x.assign(Season = combine_winning_losing_stats_for_year.Season.values))
    .pipe(lambda x:x.assign(TeamID = combine_winning_losing_stats_for_year.WTeamID.values))
    .drop(['LTeamID','win_rate'],1)
    .pipe(lambda x:x.assign(win_rate = x.winning_num_counts/(x.winning_num_counts + x.losing_num_counts)))
    .pipe(lambda x:x.assign(WFGP = x.WFGM/x.WFGA))
    .pipe(lambda x:x.assign(WFG3P = x.WFGM3/x.WFGA3))
    .pipe(lambda x:x.assign(WFTP = x.WFTM/x.WFTA))
    .pipe(lambda x:x.assign(LFGP = x.LFGM/x.LFGA))
    .pipe(lambda x:x.assign(LFG3P = x.LFGM3/x.LFGA3))
    .pipe(lambda x:x.assign(LFTP = x.LFTM/x.LFTA))
    .pipe(lambda x:x.assign(fgp = x.total_fgm/x.total_fga))
    .pipe(lambda x:x.assign(fg3p = x.total_fg3m/x.total_fg3a))
    .pipe(lambda x:x.assign(ftp = x.total_ftm/x.total_fta))
)

In [16]:
cumulative_stats_for_team_each_year.head()

,Season,WScore,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,losing_opponent_score,winning_num_counts,...,win_rate,WFGP,WFG3P,WFTP,LFGP,LFG3P,LFTP,fgp,fg3p,ftp
0,2003,825,271,480,120,259,163,249,638,12,...,0.428571,0.564583,0.463320,0.654618,0.417981,0.305556,0.647826,0.481149,0.375643,0.651357
327,2004,2229,737,1393,312,734,443,636,1709,34,...,0.607143,0.529074,0.425068,0.696541,0.408686,0.305732,0.639576,0.481886,0.378423,0.678999
653,2005,3326,1115,2180,458,1110,638,902,2533,51,...,0.600000,0.511468,0.412613,0.707317,0.408027,0.318804,0.661616,0.469388,0.373236,0.693374
982,2006,4756,1604,3171,659,1588,889,1234,3676,73,...,0.646018,0.505834,0.414987,0.720421,0.412921,0.317597,0.669456,0.472430,0.378968,0.706192
1316,2007,6347,2135,4205,870,2061,1207,1652,4844,95,...,0.664336,0.507729,0.422125,0.730630,0.412256,0.315093,0.688119,0.475389,0.384158,0.719221


In [17]:
cumulative_stats_for_team_each_year.dtypes

Season                      int64
WScore                      int64
WFGM                        int64
WFGA                        int64
WFGM3                       int64
WFGA3                       int64
WFTM                        int64
WFTA                        int64
losing_opponent_score       int64
winning_num_counts          int64
winning_opponent_score      int64
LScore                      int64
LFGM                        int64
LFGA                        int64
LFGM3                       int64
LFGA3                       int64
LFTM                        int64
LFTA                        int64
losing_num_counts           int64
total_score                 int64
total_opponent_score        int64
total_fgm                   int64
total_fga                   int64
total_fg3m                  int64
total_fg3a                  int64
total_ftm                   int64
total_fta                   int64
TeamID                      int64
win_rate                  float64
WFGP          

## Some variations to try for features
- separate winning and losing
    - reconcilation of winning and losing will have to be done later
    - could be diff between percentage --> this might give an insight of when they are losing/winning?

## Intermediate Variables
- Coach stats
    - number of years till that season
    - number of championship till that season
    - number of playoffs made till that season
    - win rate of total games till that season
        - consider regular or playoff only?
- ~~win rate for home court~~
- ~~win rate for away court~~
- ~~win rate for neutral court~~
- offensive stats
    - offensive rebounds
    - points scored
    - might try play by play later?
- defensive stats
    - defensive rebounds
    - points scored by opponents
    - turn over from play by play???
    - might try play by play later?
- blocks, steals and personal fouls


#### reconcilation of intermediate variables
- relative scoring method
     - will have a score of between 0 to 1


#### features being throw into prediction model
- test out raw intermediate variables
    - then test out difference in values
    - or something else

In [18]:
#win rate for home court
#need to ensure that the joining is from a bigger table
raw_data_regularseason.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [19]:
win_test = (
    raw_data_regularseason
    .groupby(['Season','WTeamID','WLoc'])
    .count()
    .reset_index()
    [['Season','WTeamID','WLoc','DayNum']]
)

In [20]:
lose_test = (
    raw_data_regularseason
    .groupby(['Season','LTeamID','WLoc'])
    .count()
    .reset_index()
    [['Season','LTeamID','WLoc','DayNum']]
)

In [21]:
win_test.head()

,Season,WTeamID,WLoc,DayNum
0,2003,1102,A,3
1,2003,1102,H,9
2,2003,1103,A,4
3,2003,1103,H,9
4,2003,1104,A,1


In [22]:
lose_test.head()

,Season,LTeamID,WLoc,DayNum
0,2003,1102,A,4
1,2003,1102,H,10
2,2003,1102,N,2
3,2003,1103,A,5
4,2003,1103,H,9


In [23]:
test = (
    lose_test
    .drop(['DayNum'],1)
    .append(win_test.rename(columns={"WTeamID":"LTeamID"}).drop(['DayNum'],1))
    .groupby(['Season','LTeamID','WLoc'])
    .count()
    .reset_index()
)

win_rate_type_of_court = (
    test
    .merge(win_test,how='left',left_on=['Season','LTeamID','WLoc'], right_on=['Season','WTeamID','WLoc'])
    .merge(lose_test,how='left',left_on=['Season','LTeamID','WLoc'],right_on=['Season','LTeamID','WLoc'])
    .fillna(0)
    .rename(columns={"LTeamID":"TeamID","DayNum_x":"games_won","DayNum_y":"games_lost"})
    .drop(['WTeamID'],1)
    .pipe(lambda x:x.assign(win_rate = x.games_won/(x.games_won + x.games_lost)))
)


win_rate_type_of_court.head()

,Season,TeamID,WLoc,games_won,games_lost,win_rate
0,2003,1102,A,3.0,4.0,0.428571
1,2003,1102,H,9.0,10.0,0.473684
2,2003,1102,N,0.0,2.0,0.000000
3,2003,1103,A,4.0,5.0,0.444444
4,2003,1103,H,9.0,9.0,0.500000


In [45]:
win_rate_away = (
    win_rate_type_of_court
    .query("WLoc == 'A'")
    .rename(columns={"win_rate":"win_rate_away"})
    [['Season','TeamID','win_rate_away']]
)

win_rate_neutral = (
    win_rate_type_of_court
    .query("WLoc == 'N'")
    .rename(columns={"win_rate":"win_rate_neutral"})
    [['Season','TeamID','win_rate_neutral']]
)

win_rate_home = (
    win_rate_type_of_court
    .query("WLoc == 'H'")
    .rename(columns={"win_rate":"win_rate_home"})
    [['Season','TeamID','win_rate_home']]
)

more_testing = win_rate_type_of_court.sort_values(['TeamID','Season']).query("WLoc=='A'").head().groupby(['TeamID']).cumsum()

whatever = win_rate_away.sort_values(['TeamID','Season']).head()

more_testing.pipe(lambda x:x.assign(TeamID = whatever.TeamID.values))

,Season,games_won,games_lost,win_rate,TeamID
10783,2014,0.0,5.0,0.000000,1101
11801,4029,1.0,11.0,0.142857,1101
12821,6045,3.0,16.0,0.428571,1101
13839,8062,7.0,21.0,0.873016,1101
0,2003,3.0,4.0,0.428571,1102


In [25]:
# combine back with cumulative table
cumulative_stats_for_team_each_year.head()

,Season,WScore,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,losing_opponent_score,winning_num_counts,...,win_rate,WFGP,WFG3P,WFTP,LFGP,LFG3P,LFTP,fgp,fg3p,ftp
0,2003,825,271,480,120,259,163,249,638,12,...,0.428571,0.564583,0.463320,0.654618,0.417981,0.305556,0.647826,0.481149,0.375643,0.651357
327,2004,2229,737,1393,312,734,443,636,1709,34,...,0.607143,0.529074,0.425068,0.696541,0.408686,0.305732,0.639576,0.481886,0.378423,0.678999
653,2005,3326,1115,2180,458,1110,638,902,2533,51,...,0.600000,0.511468,0.412613,0.707317,0.408027,0.318804,0.661616,0.469388,0.373236,0.693374
982,2006,4756,1604,3171,659,1588,889,1234,3676,73,...,0.646018,0.505834,0.414987,0.720421,0.412921,0.317597,0.669456,0.472430,0.378968,0.706192
1316,2007,6347,2135,4205,870,2061,1207,1652,4844,95,...,0.664336,0.507729,0.422125,0.730630,0.412256,0.315093,0.688119,0.475389,0.384158,0.719221


In [26]:
intermediate_combine_stats_for_team_each_year = (
    cumulative_stats_for_team_each_year
    .merge(win_rate_away,how='left',on=['Season','TeamID'])
    .merge(win_rate_home,how='left',on=['Season','TeamID'])
    .merge(win_rate_neutral,how='left',on=['Season','TeamID'])
)

intermediate_combine_stats_for_team_each_year.head()

,Season,WScore,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,losing_opponent_score,winning_num_counts,...,WFTP,LFGP,LFG3P,LFTP,fgp,fg3p,ftp,win_rate_away,win_rate_home,win_rate_neutral
0,2003,825,271,480,120,259,163,249,638,12,...,0.654618,0.417981,0.305556,0.647826,0.481149,0.375643,0.651357,0.428571,0.473684,0.000000
1,2004,2229,737,1393,312,734,443,636,1709,34,...,0.696541,0.408686,0.305732,0.639576,0.481886,0.378423,0.678999,1.000000,0.722222,0.500000
2,2005,3326,1115,2180,458,1110,638,902,2533,51,...,0.707317,0.408027,0.318804,0.661616,0.469388,0.373236,0.693374,0.800000,0.550000,0.500000
3,2006,4756,1604,3171,659,1588,889,1234,3676,73,...,0.720421,0.412921,0.317597,0.669456,0.472430,0.378968,0.706192,1.000000,0.736842,0.666667
4,2007,6347,2135,4205,870,2061,1207,1652,4844,95,...,0.730630,0.412256,0.315093,0.688119,0.475389,0.384158,0.719221,0.857143,0.722222,0.600000


## offensive stats

In [ ]:
# scored 
# offensive rebounds
# percentage of offensive rebounds to total rebounds
# offensive rebounding percentage, field goal missed
# defensive rebounds

In [ ]:
# block % from opponent field goal attempted
# assist / turnover ratio
# assist per fgm

# win by how many points
# lose by how many points

In [ ]:
# normalization on variables

## Features selected
- season
- region --> perhaps encode to a number. example: west - east = 1001. west = victor, east = loser
- wteamid
- wscore
- lteamid
- lscore
- wloc
- winning field goal percentage
- winning three point percentage
- winning free throw percentage
- transformed variable for rebounds (offensive and defensive)
- transformed assist
- transformed turnovers
- transformed steals
- transformed blocks
- transformed personal fouls
- repeat for losing team

*transformed variables exclude first

In [3]:
pd.read_csv("data/DataFiles/TeamCoaches.csv").head()

,Season,TeamID,FirstDayNum,LastDayNum,CoachName
0,1985,1102,0,154,reggie_minton
1,1985,1103,0,154,bob_huggins
2,1985,1104,0,154,wimp_sanderson
3,1985,1106,0,154,james_oliver
4,1985,1108,0,154,davey_whitney


In [4]:
pd.read_csv("data/DataFiles/Teams.csv")

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2018
1,1102,Air Force,1985,2018
2,1103,Akron,1985,2018
3,1104,Alabama,1985,2018
4,1105,Alabama A&M,2000,2018
5,1106,Alabama St,1985,2018
6,1107,Albany NY,2000,2018
7,1108,Alcorn St,1985,2018
8,1109,Alliant Intl,1985,1991
9,1110,American Univ,1985,2018


In [5]:
pd.read_csv("data/DataFiles/RegularSeasonCompactResults.csv").head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


In [8]:
pd.read_csv("data/DataFiles/NCAATourneyCompactResults.csv").query("DayNum==154")

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
62,1985,154,1437,66,1207,64,N,0
125,1986,154,1257,72,1181,69,N,0
188,1987,154,1231,74,1393,73,N,0
251,1988,154,1242,83,1328,79,N,0
314,1989,154,1276,80,1371,79,N,1
377,1990,154,1424,103,1181,73,N,0
440,1991,154,1181,72,1242,65,N,0
503,1992,154,1181,71,1276,51,N,0
566,1993,154,1314,77,1276,71,N,0
629,1994,154,1116,76,1181,72,N,0


,EventID,Season,DayNum,WTeamID,LTeamID,WPoints,LPoints,ElapsedSeconds,EventTeamID,EventPlayerID,EventType
0,1,2010,7,1143,1293,0,0,0,1143,600578,sub_in
1,2,2010,7,1143,1293,0,0,0,1143,600584,sub_in
2,3,2010,7,1143,1293,0,0,0,1143,600585,sub_in
3,4,2010,7,1143,1293,0,0,10,1143,600581,miss2_lay
4,5,2010,7,1143,1293,0,0,10,1143,600581,reb_off
